In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from hdfstorehelper import HDFStoreHelper
import dill
from scipy import stats
from IPython.display import display
import geopy

samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"

In [0]:
ni_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni"
imp_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40"
notimputed_vcf_gz = os.path.join(ni_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
imputed_vcf_gz = os.path.join(imp_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
vcf_files = [notimputed_vcf_gz, imputed_vcf_gz]

In [0]:
def read_df(dirname, fname):
    f = os.path.join(dirname, "%s.txt" % fname)
    return pd.read_csv(f, sep="\t", index_col=0)

In [0]:
z12_swapped = [read_df(x, "z12_swapped") for x in [ni_dir]]

In [0]:
z12_swapped[0].head()

In [0]:
assembly = "/home/cfriedline/eckertlab/projects/gypsy_moth/assemblies/masurca3/CA/10-gapclose/genome.ctg.fasta"

In [0]:
contigs_with_snps = set(["_".join(x.split("_")[0:-1]) for x in z12_swapped[0].columns])

In [0]:
outdir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3"
contig_fasta = os.path.join(outdir, "contigs_with_snps.fasta")

In [0]:
with open(contig_fasta, "w") as o:
    for rec in SeqIO.parse(assembly, 'fasta'):
        if rec.id in contigs_with_snps:
            SeqIO.write(rec, o, "fasta")

In [0]:
import pysam

In [0]:
vcf_file = pysam.VariantFile(vcf_files[1])

In [0]:
for i, elem in enumerate(z12_swapped[0].columns):
    d = elem.split("_")
    ctg = "_".join(d[0:-1])
    p = int(d[-1])
    snp = list(vcf_file.fetch(contig=ctg, start=p-1, stop=p))
    print(len(snp))
    for elem in snp:
        print(elem.chrom, elem.pos, elem.ref, elem.alts[0])
    if i == 3:
        break

In [0]:
len(z12_swapped[0].columns[0:-2])

In [0]:
pos_file = "{}.positions".format(contig_fasta)
with open(pos_file, "w") as o:
    o.write("chrom\tpos\tref\talt\n")
    for elem in z12_swapped[0].columns[0:-2]: #skip population, popid
        d = elem.split("_")
        ctg = "_".join(d[0:-1])
        p = int(d[-1])
        snp = list(vcf_file.fetch(contig=ctg, start=p-1, stop=p))
        assert len(snp) == 1
        snp = snp[0]
        assert len(snp.alts) == 1
        #print(elem.chrom, elem.pos, elem.ref, elem.alts[0])
        o.write("{}\t{}\t{}\t{}\n".format(snp.chrom, snp.pos, snp.ref, snp.alts[0]))

In [0]:
!head {pos_file}